In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import io
import urllib.request
import datetime
import gzip
import geopandas as gpd
import os
import zipfile as zf
import shapefile
from shapely.geometry import shape

Importing file with world country borders as coordinates

In [2]:
zp = zf.ZipFile('TM_WORLD_BORDERS-0.3.zip')
files_to_read = [y for y in zp.namelist() for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
dummy = zp.read(files_to_read[0])
dbf_file, prj_file, shp_file, shx_file = [zp.open(filename) for filename in files_to_read]
r = shapefile.Reader(shp = shp_file, shx = shx_file, dbf = dbf_file, encoding='windows-1252')
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]
for row in r.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names, row.record)))
#Creating a GeoDataframe of the World Borders
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry)

Scraping temperature anomlalies from NASA dataset

In [3]:
url = "https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz"
req = urllib.request.Request(url)
with gzip.open(urllib.request.urlopen(req)) as resp:
    xr_df = xr.open_dataset(io.BytesIO(resp.read()))
dfnasa = xr_df.to_dataframe()
#Transforming into pandas dataframe
dfnasa = dfnasa.reset_index()

Set year from which to obtain data and country

In [6]:
year = 1980
country= "United States"

In [11]:
def tempdata(year, country):
    #Subsetting the year range from the provided year
    tempyear = dfnasa[dfnasa['time'].dt.year >= year]
    #Creating a GeoDataFrame
    tempyear = gpd.GeoDataFrame(tempyear, geometry=gpd.points_from_xy(tempyear.lat, tempyear.lon))
    #Merging with the border data
    bord = gpd.sjoin(gdf, tempyear, how="inner")
    yearcountry = bord[bord["NAME"]==country]
    #Sorting values by time
    yearcountry = yearcountry.sort_values(by=["time"])
    yearcountry = pd.DataFrame(yearcountry)
    yearcountry = yearcountry.drop(columns= ["FIPS", "UN", "AREA","POP2005", "index_right", "nv"])
    del tempyear
    return yearcountry

In [8]:
df = tempdata(year,country)

In [15]:
df

,ISO2,ISO3,NAME,POP2005,REGION,SUBREGION,LON,LAT,geometry,lat,lon,time,tempanomaly
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-79.0,37.0,1980-01-15,0.14
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-85.0,43.0,1980-01-15,0.78
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-71.0,43.0,1980-01-15,0.43
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-75.0,41.0,1980-01-15,0.39
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-85.0,31.0,1980-01-15,0.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-83.0,37.0,2021-04-15,-3.28
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-81.0,39.0,2021-04-15,-0.78
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-79.0,43.0,2021-04-15,-0.86
208,US,USA,United States,299846449,19,21,-98.606,39.622,"MULTIPOLYGON (((-75.17029 19.93139, -75.22372 ...",-87.0,43.0,2021-04-15,-3.28


Export to csv

In [ ]:
df.to_csv("temperature_anomalies_"+str(year)+"_"+country+".csv")